# Synthetic Data Generation with LLMs (LangChain)


In [ ]:
import os

#Cell

#Synthetic Data Generation for Linux Logs Using LLMs

In [ ]:
import os
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install --upgrade --quiet pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 69.3 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade --quiet  langchain langchain_experimental langchain-openai pydantic==2.9.2 openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [langchain_experimental]


In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from pydantic import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import create_openai_data_generator
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI

In [ ]:
from typing import Optional, Union, Dict

In [ ]:
import re

def parse_log_line(line: str) -> dict:
    # Extract timestamp
    timestamp_match = re.search(r"\[(.*?)\]", line)
    timestamp = timestamp_match.group(1) if timestamp_match else ""

    # Extract time elapsed
    time_elapsed_match = re.search(r"\((.*?)\)", line)
    time_elapsed = time_elapsed_match.group(1) if time_elapsed_match else ""

    # Extract host name and event name
    parts = line.split()
    host_name = parts[2] if len(parts) > 2 else ""
    event_name = parts[3].rstrip(":") if len(parts) > 3 else ""

    # Extract all {...} blocks
    curly_groups = re.findall(r"\{(.*?)\}", line)

    # Initialize fields
    cpu_id = "0"
    process_details = ""
    kernel_details = ""

    # Extract CPU ID from first group (if present)
    if len(curly_groups) >= 1:
        cpu_match = re.search(r"cpu_id\s*=\s*(\d+)", curly_groups[0])
        cpu_id = cpu_match.group(1) if cpu_match else "0"

    # Clean and assign process/kernel details (remove quotes)
    if len(curly_groups) >= 2:
        process_details = curly_groups[1].replace('"', '').replace("'", "").strip()
    if len(curly_groups) >= 3:
        kernel_details = curly_groups[2].replace('"', '').replace("'", "").strip()

    # Build formatted example string and escape all curly braces for PromptTemplate
    formatted = (
        f"timestamp:{timestamp}, time elapsed: (+{time_elapsed}), Host Name: {host_name}, "
        f"Event Name: {event_name}, Cpu Id: {cpu_id}, process_details: {process_details}, "
        f"kernel_details: {kernel_details}"
    )

    # Escape all braces for safe PromptTemplate use
    escaped_formatted = formatted.replace("{", "{{").replace("}", "}}")

    return {"example": escaped_formatted}

In [ ]:
logs = []
with open("kernel_trace_1K.txt", "r") as file:
    for line in file:
        if line.strip():
            try:
                parsed = parse_log_line(line.strip())
                logs.append(parsed)
            except Exception as e:
                print(f"Error parsing line: {line}\n{e}")

In [ ]:
logs

[{'example': 'timestamp:06:13:02.227908688, time elapsed: (++?.?????????), Host Name: mendax, Event Name: syscall_entry_accept, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: fd = 553, upeer_addrlen = 246916502706640'},
 {'example': 'timestamp:06:13:02.227912438, time elapsed: (++0.000003750), Host Name: mendax, Event Name: syscall_exit_accept, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: ret = 576, upeer_sockaddr = 246916502706648, upeer_addrlen = 246916502706640'},
 {'example': 'timestamp:06:13:02.227914646, time elapsed: (++0.000002208), Host Name: mendax, Event Name: syscall_entry_fcntl, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: fd = 576, cmd = 3, arg = 50'},
 {'example': 'timestamp:06:13:02.227914980, time elapsed: (++0.000000334), Host Name: mendax, Event Name: syscall_exit_fcntl, Cpu Id: 2, process_details: procname = ela

In [ ]:
def extract_unique_event_examples(examples):
    seen_events = set()
    unique_examples = []

    for item in examples:
        example_text = item.get("example", "")

        # Try to extract the Event Name
        try:
            event_name = example_text.split("Event Name:")[1].split(",")[0].strip()
        except IndexError:
            continue  # Skip if malformed

        if event_name not in seen_events:
            seen_events.add(event_name)
            unique_examples.append(item)

    return unique_examples

In [ ]:
filtered_examples = extract_unique_event_examples(logs)

# Print unique event names and count
print(f"Total unique events: {len(filtered_examples)}")
for ex in filtered_examples:
    print(ex)

Total unique events: 48
{'example': 'timestamp:06:13:02.227908688, time elapsed: (++?.?????????), Host Name: mendax, Event Name: syscall_entry_accept, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: fd = 553, upeer_addrlen = 246916502706640'}
{'example': 'timestamp:06:13:02.227912438, time elapsed: (++0.000003750), Host Name: mendax, Event Name: syscall_exit_accept, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: ret = 576, upeer_sockaddr = 246916502706648, upeer_addrlen = 246916502706640'}
{'example': 'timestamp:06:13:02.227914646, time elapsed: (++0.000002208), Host Name: mendax, Event Name: syscall_entry_fcntl, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: fd = 576, cmd = 3, arg = 50'}
{'example': 'timestamp:06:13:02.227914980, time elapsed: (++0.000000334), Host Name: mendax, Event Name: syscall_exit_fcntl, Cpu Id: 2, process_detail

In [ ]:
def escape_curly_braces(examples):
    return [{"example": ex["example"].replace("{", "").replace("}", "}}")} for ex in examples]

filtered_examples = escape_curly_braces(filtered_examples)

In [ ]:
import re

def remove_high_risk_examples(examples):
    safe_examples = []
    for ex in examples:
        example_text = ex["example"]

        # Remove examples with problematic patterns
        if any([
            re.search(r"\{.*?\}", example_text),               # Unescaped curly braces
            re.search(r"\[\s*\d+\s*\]\s*=", example_text),     # Patterns like [0] =
            "fds =" in example_text,                           # Explicit fds structures
            "events =" in example_text and "POLL" in example_text  # POLLIN/POLLOUT structures
        ]):
            continue  # Skip high-risk example
        safe_examples.append(ex)

    return safe_examples

In [ ]:
filtered_examples = remove_high_risk_examples(filtered_examples)
print(f"Remaining safe examples: {len(filtered_examples)}")

Remaining safe examples: 45


In [ ]:
# Print unique event names and count
print(f"Total unique events: {len(filtered_examples)}")
for ex in filtered_examples:
    print(ex)

Total unique events: 45
{'example': 'timestamp:06:13:02.227908688, time elapsed: (++?.?????????), Host Name: mendax, Event Name: syscall_entry_accept, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: fd = 553, upeer_addrlen = 246916502706640'}
{'example': 'timestamp:06:13:02.227912438, time elapsed: (++0.000003750), Host Name: mendax, Event Name: syscall_exit_accept, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: ret = 576, upeer_sockaddr = 246916502706648, upeer_addrlen = 246916502706640'}
{'example': 'timestamp:06:13:02.227914646, time elapsed: (++0.000002208), Host Name: mendax, Event Name: syscall_entry_fcntl, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: fd = 576, cmd = 3, arg = 50'}
{'example': 'timestamp:06:13:02.227914980, time elapsed: (++0.000000334), Host Name: mendax, Event Name: syscall_exit_fcntl, Cpu Id: 2, process_detail

In [ ]:
class LinuxLog(BaseModel):
    timestamp: str  # Log timestamp
    time_elapsed: Optional[str]  # Time elapsed since the previous log
    host_name: Optional[str]  # Host name
    event_name: Optional[str]  # Event name
    cpu_id: Optional[int]  # CPU ID where the event occurred
    process_details: Optional[str] # Details object
    kernel_details: Optional[str] # Details object

In [ ]:
schema = LinuxLog.model_json_schema()

In [ ]:
schema

{'properties': {'timestamp': {'title': 'Timestamp', 'type': 'string'},
  'time_elapsed': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'title': 'Time Elapsed'},
  'host_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'title': 'Host Name'},
  'event_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'title': 'Event Name'},
  'cpu_id': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
   'title': 'Cpu Id'},
  'process_details': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'title': 'Process Details'},
  'kernel_details': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'title': 'Kernel Details'}},
 'required': ['timestamp',
  'time_elapsed',
  'host_name',
  'event_name',
  'cpu_id',
  'process_details',
  'kernel_details'],
 'title': 'LinuxLog',
 'type': 'object'}

In [ ]:
examples = [
    {
        "example": """timestamp:06:13:02.227908688, time elapsed: (+0.000002880), Host Name: mendax, Event Name: syscall_entry_accept, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: fd = 553, upeer_addrlen = 246916502706640"""
    },
    {
        "example": """timestamp:06:13:02.227912438, time elapsed: (+0.000003750), Host Name: mendax, Event Name: syscall_exit_accept, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: ret = 576, upeer_sockaddr = 246916502706648, upeer_addrlen = 246916502706640"""
    },
    {
        "example": """timestamp:06:13:02.227914646, time elapsed: (+0.000002208), Host Name: mendax, Event Name: syscall_entry_fcntl, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: fd = 576, cmd = 3, arg = 50"""
    },
    {
        "example": """timestamp:06:13:02.227914980, time elapsed: (+0.000000334), Host Name: mendax, Event Name: syscall_exit_fcntl, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: ret = 2, arg = 50"""
    },
    {
        "example": """timestamp:06:13:02.227916313, time elapsed: (+0.000001333), Host Name: mendax, Event Name: syscall_entry_getsockname, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: fd = 576, usockaddr_len = 246916502706368"""
    },
    {
        "example": """timestamp:06:13:02.227916771, time elapsed: (+0.000000458), Host Name: mendax, Event Name: syscall_exit_getsockname, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: ret = 0, usockaddr = 246916502706376, usockaddr_len = 246916502706368"""
    },
    {
        "example": """timestamp:06:13:02.227927646, time elapsed: (+0.000003292), Host Name: mendax, Event Name: syscall_entry_setsockopt, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: fd = 576, level = 6, optname = 1, optval = 246916502705868, optlen = 4"""
    },
    {
        "example": """timestamp:06:13:02.227928063, time elapsed: (+0.000000417), Host Name: mendax, Event Name: syscall_exit_setsockopt, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: ret = 0"""
    },
    {
        "example": """timestamp:06:13:02.227928771, time elapsed: (+0.000000708), Host Name: mendax, Event Name: syscall_entry_getsockopt, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: fd = 576, level = 1, optname = 7, optlen = 246916502705996"""
    },
    {
        "example": """timestamp:06:13:02.227929146, time elapsed: (+0.000000375), Host Name: mendax, Event Name: syscall_exit_getsockopt, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: ret = 0, optval = 246916502706080, optlen = 246916502705996"""
    },
    {
        "example": """timestamp:06:13:02.227938146, time elapsed: (+0.000002417), Host Name: mendax, Event Name: syscall_entry_write, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: fd = 549, buf = 246916502706248, count = 8"""
    },
    {
        "example": """timestamp:06:13:02.227940521, time elapsed: (+0.000002375), Host Name: mendax, Event Name: syscall_exit_write, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: ret = 8"""
    },
    {
        "example": """timestamp:06:13:02.227944020, time elapsed: (+0.000003499), Host Name: mendax, Event Name: syscall_entry_epoll_pwait, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: epfd = 542, maxevents = 1024, timeout = -1, sigmask = 0x0, sigsetsize = 8"""
    },
    {
        "example": """timestamp:06:13:02.227946729, time elapsed: (+0.000002709), Host Name: mendax, Event Name: syscall_exit_connect, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: ret = -115"""
    },
    {
        "example": """timestamp:06:13:02.227949604, time elapsed: (+0.000001542), Host Name: mendax, Event Name: syscall_entry_rt_sigaction, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: sig = 13, act = 281474913773656, sigsetsize = 8"""
    },
    {
        "example": """timestamp:06:13:02.227949979, time elapsed: (+0.000000375), Host Name: mendax, Event Name: syscall_exit_rt_sigaction, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: ret = 0, oact = 0"""
    },
    {
        "example": """timestamp:06:13:02.227959145, time elapsed: (+0.000003125), Host Name: mendax, Event Name: syscall_entry_futex, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: uaddr = 267960194247760, op = 129, val = 2147483647, utime = 0, uaddr2 = 203053665022808, val3 = 496169800"""
    },
    {
        "example": """timestamp:06:13:02.227961395, time elapsed: (+0.000002250), Host Name: mendax, Event Name: syscall_exit_futex, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: ret = 0, uaddr = 267960194247760, uaddr2 = 203053665022808"""
    },
    {
        "example": """timestamp:06:13:02.228003894, time elapsed: (+0.000042499), Host Name: mendax, Event Name: syscall_entry_getpid, Cpu Id: 3, process_details: procname = curl, pid = 14344, tid = 14344, kernel_details: """
    },
    {
        "example": """timestamp:06:13:02.228005269, time elapsed: (+0.000001375), Host Name: mendax, Event Name: syscall_exit_getpid, Cpu Id: 3, process_details: procname = curl, pid = 14344, tid = 14344, kernel_details: ret = 14344"""
    },
    {
        "example": """timestamp:06:13:02.228012394, time elapsed: (+0.000007125), Host Name: mendax, Event Name: syscall_entry_brk, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: brk = 203053665202176"""
    },
    {
        "example": """timestamp:06:13:02.228013435, time elapsed: (+0.000001041), Host Name: mendax, Event Name: syscall_exit_brk, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: ret = 203053665202176"""
    },
    {
        "example": """timestamp:06:13:02.228181389, time elapsed: (+0.000006833), Host Name: mendax, Event Name: syscall_entry_getrandom, Cpu Id: 3, process_details: procname = curl, pid = 14344, tid = 14344, kernel_details: buf = 222501009261744, count = 48, flags = 0"""
    },
    {
        "example": """timestamp:06:13:02.228182306, time elapsed: (+0.000000917), Host Name: mendax, Event Name: syscall_exit_getrandom, Cpu Id: 3, process_details: procname = curl, pid = 14344, tid = 14344, kernel_details: ret = 48, buf = 222501009261744, count = 48, flags = 0"""
    },
    {
        "example": """timestamp:06:13:02.228694417, time elapsed: (+0.000019583), Host Name: mendax, Event Name: syscall_entry_munmap, Cpu Id: 3, process_details: procname = gnome-shell, pid = 3154, tid = 3154, kernel_details: addr = 248270898528256, len = 7602176"""
    },
    {
        "example": """timestamp:06:13:02.228704875, time elapsed: (+0.000010458), Host Name: mendax, Event Name: syscall_exit_munmap, Cpu Id: 3, process_details: procname = gnome-shell, pid = 3154, tid = 3154, kernel_details: ret = 0"""
    },
    {
        "example": """timestamp:06:13:02.228708583, time elapsed: (+0.000003708), Host Name: mendax, Event Name: syscall_entry_sendto, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: fd = 4, buff = 203053665469200, len = 517, flags = 16384, addr = 0, addr_len = 0"""
    },
    {
        "example": """timestamp:06:13:02.228715167, time elapsed: (+0.000006584), Host Name: mendax, Event Name: syscall_exit_sendto, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: ret = 517"""
    },
    {
        "example": """timestamp:06:13:02.228717458, time elapsed: (+0.000002291), Host Name: mendax, Event Name: syscall_entry_recvfrom, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: fd = 4, size = 5, flags = 0, addr_len = 0"""
    },
    {
        "example": """timestamp:06:13:02.228718250, time elapsed: (+0.000000792), Host Name: mendax, Event Name: syscall_exit_recvfrom, Cpu Id: 2, process_details: procname = curl, pid = 14345, tid = 14345, kernel_details: ret = -11, ubuf = 203053665491603, addr = 0, addr_len = 0"""
    },
]

In [ ]:
for ex in examples:
    print(ex["example"])

timestamp:06:13:02.227908688, time elapsed: (+0.000002880), Host Name: mendax, Event Name: syscall_entry_accept, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: fd = 553, upeer_addrlen = 246916502706640
timestamp:06:13:02.227912438, time elapsed: (+0.000003750), Host Name: mendax, Event Name: syscall_exit_accept, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: ret = 576, upeer_sockaddr = 246916502706648, upeer_addrlen = 246916502706640
timestamp:06:13:02.227914646, time elapsed: (+0.000002208), Host Name: mendax, Event Name: syscall_entry_fcntl, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: fd = 576, cmd = 3, arg = 50
timestamp:06:13:02.227914980, time elapsed: (+0.000000334), Host Name: mendax, Event Name: syscall_exit_fcntl, Cpu Id: 2, process_details: procname = elasticsearch, pid = 11822, tid = 11859, kernel_details: ret = 2, arg = 50
tim

In [ ]:
all_synthetic_results = []

# ✅ Preprocess examples once (escape curly braces)
escaped_examples = []
for ex in filtered_examples[:15]:
    escaped_example = ex["example"].replace("{", "{{").replace("}", "}}")
    escaped_examples.append({"example": escaped_example})

In [ ]:
escaped_examples

[{'example': 'timestamp:06:13:02.227908688, time elapsed: (++?.?????????), Host Name: mendax, Event Name: syscall_entry_accept, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: fd = 553, upeer_addrlen = 246916502706640'},
 {'example': 'timestamp:06:13:02.227912438, time elapsed: (++0.000003750), Host Name: mendax, Event Name: syscall_exit_accept, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: ret = 576, upeer_sockaddr = 246916502706648, upeer_addrlen = 246916502706640'},
 {'example': 'timestamp:06:13:02.227914646, time elapsed: (++0.000002208), Host Name: mendax, Event Name: syscall_entry_fcntl, Cpu Id: 2, process_details: procname = elasticsearch[m, pid = 11822, tid = 11859, kernel_details: fd = 576, cmd = 3, arg = 50'},
 {'example': 'timestamp:06:13:02.227914980, time elapsed: (++0.000000334), Host Name: mendax, Event Name: syscall_exit_fcntl, Cpu Id: 2, process_details: procname = ela

In [ ]:
# Run generation loop
for i in range(2):
    print(f"[Batch {i+1}/2] Generating...")

    OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

    prompt_template = FewShotPromptTemplate(
        prefix=SYNTHETIC_FEW_SHOT_PREFIX,
        examples=examples,
        suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
        input_variables=["subject", "extra"],
        example_prompt=OPENAI_TEMPLATE,
    )

    synthetic_data_generator = create_openai_data_generator(
        output_schema=LinuxLog,
        llm=ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=1,
            api_key=os.environ["OPENAI_API_KEY"]
        ),
        prompt=prompt_template,
    )

    synthetic_results = synthetic_data_generator.generate(
        subject="Linux Logs using LLTNG tool",
        extra="the host_name must be synthetic-host.",
        runs=500,
        safe=True,  # silently skips invalid outputs
    )

    all_synthetic_results.extend(synthetic_results)

print(f"✅ Total synthetic logs generated: {len(all_synthetic_results)}")

[Batch 1/2] Generating...
[Batch 2/2] Generating...
✅ Total synthetic logs generated: 1000


In [ ]:
all_synthetic_results

[LinuxLog(timestamp='06:13:02.227908688', time_elapsed='(+0.000002880)', host_name='synthetic-host', event_name='syscall_entry_accept', cpu_id=2, process_details='procname = elasticsearch, pid = 11822, tid = 11859', kernel_details='fd = 553, upeer_addrlen = 246916502706640'),
 LinuxLog(timestamp='06:13:02.227908688', time_elapsed='(+0.000002880)', host_name='synthetic-host', event_name='syscall_entry_accept', cpu_id=2, process_details='procname = elasticsearch, pid = 11822, tid = 11859', kernel_details='fd = 553, upeer_addrlen = 246916502706640'),
 LinuxLog(timestamp='06:13:02.227908688', time_elapsed='(+0.000002880)', host_name='synthetic-host', event_name='syscall_entry_accept', cpu_id=2, process_details='procname = elasticsearch, pid = 11822, tid = 11859', kernel_details='fd = 553, upeer_addrlen = 246916502706640'),
 LinuxLog(timestamp='06:13:02.227914980', time_elapsed='(+0.000000334)', host_name='synthetic-host', event_name='syscall_exit_fcntl', cpu_id=2, process_details='procname

In [ ]:
with open("synthetic_logs.txt", "w") as f:
    for item in all_synthetic_results:
        f.write(str(item) + "\n")

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.DataFrame([result.dict() for result in all_synthetic_results])

In [ ]:
dataset.head(1000)

,timestamp,time_elapsed,host_name,event_name,cpu_id,process_details,kernel_details
0,06:13:02.227908688,(+0.000002880),synthetic-host,syscall_entry_accept,2,"procname = elasticsearch, pid = 11822, tid = 1...","fd = 553, upeer_addrlen = 246916502706640"
1,06:13:02.227908688,(+0.000002880),synthetic-host,syscall_entry_accept,2,"procname = elasticsearch, pid = 11822, tid = 1...","fd = 553, upeer_addrlen = 246916502706640"
2,06:13:02.227908688,(+0.000002880),synthetic-host,syscall_entry_accept,2,"procname = elasticsearch, pid = 11822, tid = 1...","fd = 553, upeer_addrlen = 246916502706640"
3,06:13:02.227914980,(+0.000000334),synthetic-host,syscall_exit_fcntl,2,"procname = elasticsearch, pid = 11822, tid = 1...","ret = 2, arg = 50"
4,06:13:02.227914980,(+0.000000334),synthetic-host,syscall_exit_fcntl,2,"procname = elasticsearch, pid = 11822, tid = 1...","ret = 2, arg = 50"
...,...,...,...,...,...,...,...
995,06:13:02.227959145,+0.000003125,synthetic-host,syscall_entry_futex,2,"procname = curl, pid = 14345, tid = 14345","uaddr = 267960194247760, op = 129, val = 21474..."
996,06:13:02.228694417,+0.000019583,synthetic-host,syscall_entry_munmap,3,"procname = gnome-shell, pid = 3154, tid = 3154","addr = 248270898528256, len = 7602176"
997,06:13:02.228694417,+0.000019583,synthetic-host,syscall_entry_munmap,3,"procname = gnome-shell, pid = 3154, tid = 3154","addr = 248270898528256, len = 7602176"
998,06:12:22.987654321,+0.000042499,synthetic-host,syscall_entry_accept,2,"procname = apache, pid = 12345, tid = 12345","fd = 567, upeer_addrlen = 246916502706640"


In [ ]:
dataset.to_csv("synthetic_logs_llm.csv", index=False)

In [ ]:
import pandas as pd
import re

def parse_key_value_block(block):
    """
    Parse a block like 'key1 = val1, key2 = val2' into a dictionary.
    Handles quoted values and removes extra whitespace.
    """
    pattern = re.findall(r'(\w+)\s*=\s*("[^"]*"|[^,]+)', block)
    return {
      key: re.sub(r"\[.*", "", val.strip('"').strip()) if key == "procname" else val.strip('"').strip()
      for key, val in pattern
  }

def parse_lttng_logs(file_path):
    log_pattern = re.compile(
        r"\[(?P<timestamp>[\d:.]+)\] "
        r"\(\+(?P<time_elapsed>[\d.?\?]+)\) "
        r"(?P<host_name>[\w\-.]+) "
        r"(?P<event_name>[\w_]+): "
        r"\{ cpu_id = (?P<cpu_id>\d+) \}, "
        r"\{ (?P<process_details>.*?) \}, "
        r"\{ (?P<kernel_details>.*?) \}"
    )

    records = []

    with open(file_path, 'r') as file:
        for line in file:
            match = log_pattern.match(line.strip())
            if match:
                base = match.groupdict()

                # Flatten process and kernel details
                process_fields = parse_key_value_block(base.pop("process_details"))
                kernel_fields = parse_key_value_block(base.pop("kernel_details"))

                # Combine all fields
                combined = {**base, **process_fields, **kernel_fields}
                records.append(combined)

    return pd.DataFrame(records)

file_path = "kernel_trace_1K.txt"
real_data = parse_lttng_logs(file_path)

In [ ]:
real_data.head(1000)

,timestamp,time_elapsed,host_name,event_name,cpu_id,procname,pid,tid,fd,upeer_addrlen,...,ubuf,op_enum,event,msg,offset,nbytes,advice,ufd,utmr,otmr
0,06:13:02.227908688,?.?????????,mendax,syscall_entry_accept,2,elasticsearch,11822,11859,553,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06:13:02.227912438,0.000003750,mendax,syscall_exit_accept,2,elasticsearch,11822,11859,NaN,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06:13:02.227914646,0.000002208,mendax,syscall_entry_fcntl,2,elasticsearch,11822,11859,576,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06:13:02.227914980,0.000000334,mendax,syscall_exit_fcntl,2,elasticsearch,11822,11859,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06:13:02.227916313,0.000001333,mendax,syscall_entry_getsockname,2,elasticsearch,11822,11859,576,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,06:13:02.232285404,0.000000250,mendax,syscall_exit_sync_file_range,0,lttng-consumerd,14323,14330,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
966,06:13:02.232285613,0.000000209,mendax,syscall_entry_fadvise64_64,0,lttng-consumerd,14323,14330,35,NaN,...,NaN,NaN,NaN,NaN,143360,NaN,4,NaN,NaN,NaN
967,06:13:02.232286446,0.000000833,mendax,syscall_exit_fadvise64_64,0,lttng-consumerd,14323,14330,35,NaN,...,NaN,NaN,NaN,NaN,143360,NaN,4,NaN,NaN,NaN
968,06:13:02.232286779,0.000000333,mendax,syscall_entry_ioctl,0,lttng-consumerd,14323,14330,34,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import re

def parse_key_value_block(block):
    if pd.isna(block):
        return {}
    # Matches key = value pairs, accounting for values with spaces/quotes
    pairs = re.findall(r'(\w+)\s*=\s*("[^"]+"|[^,]+)', block)
    return {
        key.strip(): re.sub(r"\[.*", "", val.strip().strip('"')) if key == "procname" else val.strip().strip('"')
        for key, val in pairs
    }

# Apply parser to both columns and concatenate
def flatten_details_columns(df):
    process_df = df['process_details'].apply(parse_key_value_block).apply(pd.Series)
    kernel_df = df['kernel_details'].apply(parse_key_value_block).apply(pd.Series)

    # Drop the original detail columns and join new parsed ones
    df = df.drop(columns=['process_details', 'kernel_details'])
    df = pd.concat([df, process_df.add_prefix("proc_"), kernel_df.add_prefix("kern_")], axis=1)
    return df

# Apply to your DataFrame
synthetic_data = flatten_details_columns(dataset)

In [ ]:
synthetic_data.head(1000)

,timestamp,time_elapsed,host_name,event_name,cpu_id,proc_procname,proc_pid,proc_tid,kern_fd,kern_upeer_addrlen,...,kern_brk,kern_buff,kern_addr_len,kern_size,kern_ubuf,kern_path,kern_cmd,kern_domain,kern_type,kern_protocol
0,06:13:02.227908688,(+0.000002880),synthetic-host,syscall_entry_accept,2,elasticsearch,11822,11859,553,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06:13:02.227908688,(+0.000002880),synthetic-host,syscall_entry_accept,2,elasticsearch,11822,11859,553,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06:13:02.227908688,(+0.000002880),synthetic-host,syscall_entry_accept,2,elasticsearch,11822,11859,553,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06:13:02.227914980,(+0.000000334),synthetic-host,syscall_exit_fcntl,2,elasticsearch,11822,11859,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06:13:02.227914980,(+0.000000334),synthetic-host,syscall_exit_fcntl,2,elasticsearch,11822,11859,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,06:13:02.227959145,+0.000003125,synthetic-host,syscall_entry_futex,2,curl,14345,14345,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,06:13:02.228694417,+0.000019583,synthetic-host,syscall_entry_munmap,3,gnome-shell,3154,3154,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,06:13:02.228694417,+0.000019583,synthetic-host,syscall_entry_munmap,3,gnome-shell,3154,3154,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,06:12:22.987654321,+0.000042499,synthetic-host,syscall_entry_accept,2,apache,12345,12345,567,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
synthetic_df = synthetic_data.copy(deep=True)
real_df = real_data.copy(deep=True)

In [ ]:
# Converting timestamps to numerical values (seconds since epoch)
real_df['timestamp'] = pd.to_datetime(real_df['timestamp']).astype(int) / 10**9
synthetic_df['timestamp'] = pd.to_datetime(synthetic_df['timestamp']).astype(int) / 10**9

<ipython-input-70-350e5b37e7b5>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  real_df['timestamp'] = pd.to_datetime(real_df['timestamp']).astype(int) / 10**9
<ipython-input-70-350e5b37e7b5>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  synthetic_df['timestamp'] = pd.to_datetime(synthetic_df['timestamp']).astype(int) / 10**9


In [ ]:
# Remove parentheses and '+' from the 'time_elapsed' column
synthetic_df['time_elapsed'] = synthetic_df['time_elapsed'].str.replace(r"[()+]", "", regex=True)

In [ ]:
synthetic_df.rename(columns={"proc_procname": "procname"}, inplace=True)

In [ ]:
synthetic_df

,timestamp,time_elapsed,host_name,event_name,cpu_id,procname,proc_pid,proc_tid,kern_fd,kern_upeer_addrlen,...,kern_buff,kern_addr_len,kern_size,kern_ubuf,kern_path,kern_cmd,kern_domain,kern_type,kern_protocol,label
0,1.746426e+09,2.880000e-06,1,0,2,elasticsearch,11822,11859,553,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1.746426e+09,2.880000e-06,1,0,2,elasticsearch,11822,11859,553,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1.746426e+09,2.880000e-06,1,0,2,elasticsearch,11822,11859,553,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1.746426e+09,3.340000e-07,1,24,2,elasticsearch,11822,11859,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1.746426e+09,3.340000e-07,1,24,2,elasticsearch,11822,11859,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.746426e+09,3.125000e-06,1,48,2,curl,14345,14345,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
996,1.746426e+09,1.958300e-05,1,51,3,gnome-shell,3154,3154,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
997,1.746426e+09,1.958300e-05,1,51,3,gnome-shell,3154,3154,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
998,1.746426e+09,4.249900e-05,1,0,2,apache,12345,12345,567,246916502706640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# Converting the time_elapsed to numeric (handle missing values)
real_df['time_elapsed'] = pd.to_numeric(real_df['time_elapsed'].str.replace('?', 'nan'), errors='coerce').fillna(0)
synthetic_df['time_elapsed'] = pd.to_numeric(synthetic_df['time_elapsed'].str.replace('?', 'nan'), errors='coerce').fillna(0)

In [ ]:
real_df["cpu_id"] = pd.to_numeric(real_df["cpu_id"], errors="coerce")

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import wasserstein_distance
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score
from scipy.stats import chi2_contingency

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Convert categorical columns to strings before encoding
categorical_columns = ["host_name", "event_name", "procname"]
for col in categorical_columns:
    real_df[col] = real_df[col].astype(str)
    synthetic_df[col] = synthetic_df[col].astype(str)

    encoder = LabelEncoder()

    # Fit encoder on combined unique values
    encoder.fit(pd.concat([real_df[col], synthetic_df[col]], axis=0).astype(str))

    # Transform both datasets
    real_df[col] = encoder.transform(real_df[col])
    synthetic_df[col] = encoder.transform(synthetic_df[col])


In [ ]:
# ----- 1. Basic Statistical Comparison -----
print("\n--- Basic Statistical Comparison ---")
print("\nReal Data Summary:\n", real_df.describe())
print("\nSynthetic Data Summary:\n", synthetic_df.describe())


--- Basic Statistical Comparison ---

Real Data Summary:
           timestamp  time_elapsed  host_name  event_name      cpu_id  \
count  9.700000e+02  9.700000e+02      970.0  970.000000  970.000000   
mean   1.746426e+09  4.158827e-06        0.0   28.036082    0.588660   
std    1.290041e-03  2.475477e-05        0.0   12.923995    1.066845   
min    1.746426e+09  0.000000e+00        0.0    0.000000    0.000000   
25%    1.746426e+09  2.080000e-07        0.0   19.000000    0.000000   
50%    1.746426e+09  3.330000e-07        0.0   25.000000    0.000000   
75%    1.746426e+09  1.000000e-06        0.0   40.000000    1.000000   
max    1.746426e+09  5.419020e-04        0.0   52.000000    3.000000   

         procname  label  
count  970.000000  970.0  
mean    13.112371    1.0  
std      3.709073    0.0  
min      0.000000    1.0  
25%     15.000000    1.0  
50%     15.000000    1.0  
75%     15.000000    1.0  
max     18.000000    1.0  

Synthetic Data Summary:
           timestamp  ti

Timestamps in both datasets are close, but the synthetic data has a wider range, suggesting it may not fully match the real data’s time distribution. <br>Event distributions are somewhat similar, but the synthetic dataset has a slightly higher mean (10.2 vs. 8.96) and a lower max value (15 vs. 19), indicating possible missing event types.
<br>
CPU ID values differ significantly—real data has only one unique value (0), while the synthetic dataset varies across multiple CPUs (0-4), suggesting it introduces different CPU distributions.

# Wasserstein Distance:

Measures how far apart the numerical distributions are. <br>
Smaller values mean better similarity.

\
\begin{array}{|c|c|l|}
\hline
\textbf{Feature} & \textbf{Wasserstein Distance} & \textbf{Interpretation} \\
\hline
\textbf{Timestamp} & \textbf{2766.78} & \textbf{High difference} : \textbf{synthetic timestamps deviate significantly} \\
\hline
\textbf{Time Elapsed} & \textbf{2.85e-05} & \textbf{Very close match} : \textbf{synthetic data closely follows real timing intervals} \\
\hline
\textbf{CPU ID} & \textbf{0.77} & \textbf{Moderate deviation} : \textbf{synthetic CPU usage does not match real distribution} \\
\hline
\end{array}



In [ ]:
# ----- 2. Wasserstein Distance -----
print("\n--- Wasserstein Distance (Numerical Feature Distributions) ---")
for col in ["timestamp", "time_elapsed", "cpu_id"]:
    dist = wasserstein_distance(real_df[col], synthetic_df[col])
    print(f"Wasserstein Distance for {col}: {dist}")


--- Wasserstein Distance (Numerical Feature Distributions) ---
Wasserstein Distance for timestamp: 48.56541098290869
Wasserstein Distance for time_elapsed: 6.983278999999994e-06
Wasserstein Distance for cpu_id: 1.6803402061855668


Trains a Random Forest Classifier to distinguish real vs synthetic logs.<br>
AUC Score near 0.5 means synthetic data is similar to real data.<br>
AUC Score closer to 1 means synthetic logs are too different.<br>

In [ ]:
# ----- 4. Domain Classifier Test -----
print("\n--- Domain Classifier Test (Discriminability) ---")
real_df["label"] = 1  # Real data label
synthetic_df["label"] = 0  # Synthetic data label
combined_df = pd.concat([real_df, synthetic_df])


--- Domain Classifier Test (Discriminability) ---


In [ ]:
# Features for classification
features = ["timestamp", "time_elapsed", "cpu_id", "host_name", "event_name"]
X = combined_df[features]
y = combined_df["label"]

In [ ]:
# Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a classifier (Random Forest)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Evaluate classifier
y_pred = clf.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred)
print(f"AUC Score (closer to 0.5 means high similarity): {auc_score}")

AUC Score (closer to 0.5 means high similarity): 1.0


#Anomaly Detection Test:

Trains Isolation Forest on real data and tests anomalies in synthetic data.<br>
Higher % of anomalies → Poor quality synthetic data.<br>

In [ ]:
# ----- 5. Anomaly Detection Test -----
print("\n--- Anomaly Detection Test ---")
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(real_df[features])  # Train on real logs

# Predict anomalies in synthetic logs
synthetic_anomaly_scores = iso_forest.decision_function(synthetic_df[features])
anomalies = np.mean(synthetic_anomaly_scores < -0.1) * 100  # % of anomalies
print(f"Percentage of anomalies in synthetic data: {anomalies:.2f}%")


--- Anomaly Detection Test ---
Percentage of anomalies in synthetic data: 1.50%
